## Data Fetching Examples

### 1- Import the DataFetcher

In [2]:
from torchtrade.data_processors.ccxt_fetcher import CcxtFetcher

### 2 - Create DataFetcher instance

DateFetcher Class accepts the following arguments
- source (str): The exchange source to fetch data from (e.g binance)
- symbols (list): List of symbols to fetch data for (e.g ["BTC/USDT","ETH/USDT"])
- timeframe (str): Timeframe of data to fetch (e.g "4h")
- since (datetime, optional): The start date of the data to fetch. Defaults to 2015/01/01.
- until (datetime, optional): The end date of the data to fetch. Defaults to None. 
  if until is none, it's automatically set to today's date

In [3]:
from datetime import datetime
since = datetime(2020,1,1)
until = datetime(2023,1,1)
datafetcher = DataFetcher("binance",["BTC/USDT","ETH/USDT","ADA/USDT"],"1d", since = since , until = until)

### 3 - Fetching Data
-- WORK IN PROGRESS

In [4]:
data = datafetcher.fetch_data()
data

Fetching 1d timeframe data since 2020-01-01 00:00:00 until 2023-01-01 00:00:00


isTraded     open    high      low    close  \
symbol   timestamp                                                   
BTC/USDT 1577836800000        1  7195.24  7255.0  7175.15  7200.85   
         1577923200000        1  7200.77  7212.5  6924.74  6965.71   
         1578009600000        1  6965.49  7405.0  6871.04  7344.96   
         1578096000000        1   7345.0  7404.0  7272.21  7354.11   
         1578182400000        1  7354.19  7495.0   7318.0  7358.75   
...                         ...      ...     ...      ...      ...   
ADA/USDT 1672185600000        1   0.2603  0.2611   0.2468   0.2482   
         1672272000000        1   0.2483  0.2484   0.2396    0.244   
         1672358400000        1    0.244  0.2463   0.2392   0.2458   
         1672444800000        1   0.2457  0.2521    0.244   0.2458   
         1672531200000        1   0.2458    0.25   0.2434   0.2499   

                              volume  
symbol   timestamp                    
BTC/USDT 1577836800000  16792.388165  
         1577923200000  31951.483932  
         1578009600000  68428.500451  
         1578096000000  29987.974977  
         1578182400000  38331.085604  
...                              ...  
ADA/USDT 1672185600000   115688916.7  
         1672272000000   119235796.6  
         1672358400000    77783257.0  
         1672444800000    66923069.1  
         1672531200000    49487029.5  

[3291 rows x 6 columns]